In [1]:
from zammad_py import ZammadAPI
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv()

True

In [2]:
# Initialize the client with the URL, username, and password
# Note the Host URL should be in this format: 'https://zammad.example.org/api/v1/'
url = os.getenv('ZAMMAD_URL')
password = os.getenv('ZAMMAD_PASSWORD')
user_name = os.getenv('ZAMMAD_USERNAME')
token = os.getenv('ZAMMAD_HTTP_TOKEN')
client = ZammadAPI(url=url, username=user_name, password=password,http_token=token)
 

In [3]:
# Example: Access all users
this_page = client.user.all()
# Iterate through page object
for user in this_page:
    print(user)

# Example: Get information about the current user
client.user.me()['firstname']

{'id': 1, 'organization_id': None, 'login': '-', 'firstname': '-', 'lastname': '', 'email': '', 'image': None, 'image_source': None, 'web': '', 'phone': '', 'fax': '', 'mobile': '', 'department': '', 'street': '', 'zip': '', 'city': '', 'country': '', 'address': '', 'vip': False, 'verified': False, 'active': False, 'note': '', 'last_login': None, 'source': None, 'login_failed': 0, 'out_of_office': False, 'out_of_office_start_at': None, 'out_of_office_end_at': None, 'out_of_office_replacement_id': None, 'preferences': {}, 'updated_by_id': 1, 'created_by_id': 1, 'created_at': '2025-07-17T04:43:54.217Z', 'updated_at': '2025-07-17T04:43:54.217Z', 'role_ids': [], 'two_factor_preference_ids': [], 'organization_ids': [], 'authorization_ids': [], 'overview_sorting_ids': [], 'group_ids': {}, 'roles': [], 'two_factor_preferences': [], 'organizations': [], 'authorizations': [], 'overview_sortings': [], 'groups': {}, 'created_by': '-', 'updated_by': '-'}
{'id': 3, 'organization_id': 2, 'login': 'l

'Qwerasdf'

In [4]:
tickets = client.ticket.all()
print(f"Retrieved {len(tickets)} tickets from the first page.")
for ticket in tickets:
    print(f"Ticket ID: {ticket['id']}, Title: {ticket['title']}")

Retrieved 9 tickets from the first page.
Ticket ID: 1, Title: Welcome to Zammad!
Ticket ID: 2, Title: Test Ticket!
Ticket ID: 3, Title: vpn prblem
Ticket ID: 8, Title: VPN drops intermittently
Ticket ID: 9, Title: group problem
Ticket ID: 10, Title: group_ticket is not working with time and have a solution
Ticket ID: 11, Title: group_ticket is not working with time and have a solution
Ticket ID: 12, Title: grequie high  download data with without resolution
Ticket ID: 13, Title: wifi is not working


In [5]:
import json
from zammad_py import ZammadAPI

try:
    print("Attempting to retrieve all tickets...")
    # client.ticket.all() returns a Pagination object
    pagination_object = client.ticket.all()

    # Convert the Pagination object into a list of dictionaries
    all_tickets_list = list(pagination_object)

    if not all_tickets_list:
        print("No tickets found.")
    else:
        print(f"Retrieved {len(all_tickets_list)} tickets.")

        # Convert the list of ticket dictionaries into a single JSON array string
        # Using indent=2 for pretty-printing, remove it for a single-line minified JSON.
        all_tickets_json = json.dumps(all_tickets_list, indent=2)

        print("\n--- Raw JSON Structure of ALL Tickets ---")
        print(all_tickets_json)
        print("\n--- End of Raw JSON Structure ---")

except Exception as e:
    print(f"An error occurred: {e}")

Attempting to retrieve all tickets...
Retrieved 9 tickets.

--- Raw JSON Structure of ALL Tickets ---
[
  {
    "id": 1,
    "group_id": 1,
    "priority_id": 2,
    "state_id": 1,
    "organization_id": 1,
    "number": "91001",
    "title": "Welcome to Zammad!",
    "owner_id": 1,
    "customer_id": 2,
    "note": null,
    "first_response_at": null,
    "first_response_escalation_at": null,
    "first_response_in_min": null,
    "first_response_diff_in_min": null,
    "close_at": null,
    "close_escalation_at": null,
    "close_in_min": null,
    "close_diff_in_min": null,
    "update_escalation_at": null,
    "update_in_min": null,
    "update_diff_in_min": null,
    "last_close_at": null,
    "last_contact_at": "2025-06-27T12:46:06.972Z",
    "last_contact_agent_at": null,
    "last_contact_customer_at": "2025-06-27T12:46:06.972Z",
    "last_owner_update_at": null,
    "create_article_type_id": 5,
    "create_article_sender_id": 2,
    "article_count": 1,
    "escalation_at": nul

In [13]:

customer_email = "ckhainar140@gmail.com"
customer_firstname = "Chetana"
customer_lastname = "karnar"

try:
    # 1. Try to find the customer by email using the Zammad search query syntax
    print(f"Searching for customer: {customer_email}...")

    # The Zammad API's search endpoint uses a 'query' parameter
    # The zammad_py client's search method expects this query string directly as its argument.
    # For an exact email match, it's best to quote it.
    search_query_string = f'email:"{customer_email}"'

    # The search method returns a Pagination object
    existing_users_pagination = client.user.search(search_query_string)
    
    # Convert the Pagination object to a list to iterate
    existing_customers = list(existing_users_pagination)

    customer_id = None
    if existing_customers:
        # Even with an exact query, Zammad's search might return partial matches
        # or multiple results if the email is part of other fields.
        # It's safest to filter locally for an exact email match.
        exact_match_customer = next((user for user in existing_customers if user.get('email') == customer_email), None)

        if exact_match_customer:
            customer_id = exact_match_customer['id']
            print(f"Customer '{customer_email}' already exists with ID: {customer_id}")
        else:
            print(f"Customer '{customer_email}' not found via exact email match in search results.")
    else:
        print(f"No customers found matching '{customer_email}'.")


    if customer_id is None: # If not found after search (or if no exact match)
        # 2. If customer not found, create a new one
        print(f"Customer '{customer_email}' not found. Creating new customer...")
        new_customer_params = {
            "email": customer_email,
            "firstname": customer_firstname,
            "lastname": customer_lastname,
            "roles": ["Customer"] # Assign the 'Customer' role
        }
        new_customer = client.user.create(params=new_customer_params)
        customer_id = new_customer['id']
        print(f"New customer '{customer_email}' successfully created with ID: {customer_id}")
        print("\n--- Full JSON of Created Customer ---")
        print(json.dumps(new_customer, indent=2))
        print("--- End of Full JSON ---")
    else:
        print(f"No new customer created as '{customer_email}' already exists.")

except Exception as e:
    print(f"An error occurred: {e}")

Searching for customer: ckhainar140@gmail.com...
Customer 'ckhainar140@gmail.com' already exists with ID: 8
No new customer created as 'ckhainar140@gmail.com' already exists.


In [ ]:



try:
    # --- Part 1: Ensure Customer Exists (Search or Create) ---
    print(f"Searching for customer: {customer_email}...")

    search_query_string = f'email:"{customer_email}"'
    existing_users_pagination = client.user.search(search_query_string)
    existing_customers = list(existing_users_pagination)

    customer_id = None
    if existing_customers:
        exact_match_customer = next((user for user in existing_customers if user.get('email') == customer_email), None)

        if exact_match_customer:
            customer_id = exact_match_customer['id']
            print(f"Customer '{customer_email}' already exists with ID: {customer_id}")
        else:
            print(f"Customer '{customer_email}' not found via exact email match in search results.")
    else:
        print(f"No customers found matching '{customer_email}'.")


    if customer_id is None:
        print(f"Customer '{customer_email}' not found. Creating new customer...")
        new_customer_params = {
            "email": customer_email,
            "firstname": customer_firstname,
            "lastname": customer_lastname,
            "roles": ["Customer"]
        }
        new_customer = client.user.create(params=new_customer_params)
        customer_id = new_customer['id']
        print(f"New customer '{customer_email}' successfully created with ID: {customer_id}")
    else:
        print(f"Proceeding with existing customer ID: {customer_id}")

    # --- Part 2: Create the Ticket using the obtained Customer ID ---
    if customer_id:
        ticket_params = {
            "title": "grequie high  download data with without resolution",
            "group_id": 1,
            "customer_id": customer_id,
            "article": {
                "subject": "requie high  download data",
                "body": " when i download small video it reuiquies lot of amount and the applicaation does not have any resolution option by wwhich i can changes the quality of the video.",
                "type": "email",
                "internal": False,
                # --- THIS IS THE CRITICAL ADDITION ---
                "to": customer_email
            }
        }

        print("\nAttempting to create a new ticket with the customer...")
        ticket = client.ticket.create(params=ticket_params)

        print(f"Successfully created ticket with ID: {ticket['id']}")
        print(f"Ticket Number: {ticket['number']}")
        print(f"Ticket Title: {ticket['title']}")

        print("\n--- Full JSON of Created Ticket ---")
        print(json.dumps(ticket, indent=2))
        print("--- End of Full JSON ---")
    else:
        print("Could not obtain a customer ID. Skipping ticket creation.")

except Exception as e:
    print(f"An error occurred: {e}")

Searching for customer: bhamratanveer00612@gmail.com...
Customer 'bhamratanveer00612@gmail.com' already exists with ID: 7
Proceeding with existing customer ID: 7

Attempting to create a new ticket with the customer...
Successfully created ticket with ID: 12
Ticket Number: 91008
Ticket Title: grequie high  download data with without resolution

--- Full JSON of Created Ticket ---
{
  "id": 12,
  "group_id": 1,
  "priority_id": 2,
  "state_id": 2,
  "organization_id": null,
  "number": "91008",
  "title": "grequie high  download data with without resolution",
  "owner_id": 1,
  "customer_id": 7,
  "note": null,
  "first_response_at": "2025-07-04T05:35:09.696Z",
  "first_response_escalation_at": null,
  "first_response_in_min": null,
  "first_response_diff_in_min": null,
  "close_at": null,
  "close_escalation_at": null,
  "close_in_min": null,
  "close_diff_in_min": null,
  "update_escalation_at": null,
  "update_in_min": null,
  "update_diff_in_min": null,
  "last_close_at": null,
  "la

In [12]:

try:
    print("Attempting to retrieve all Zammad Groups (Departments)...")

    # Get all groups - this returns a Pagination object
    all_groups_pagination = client.group.all()

    # Convert the Pagination object to a list of group dictionaries
    all_groups_list = list(all_groups_pagination)

    if not all_groups_list:
        print("No groups found in your Zammad instance.")
    else:
        print(f"Retrieved {len(all_groups_list)} groups.")
        print("\n--- List of Zammad Groups ---")
        for group in all_groups_list:
            # Each 'group' is a dictionary. Common fields are 'id' and 'name'.
            print(f"Group ID: {group.get('id')}, Name: {group.get('name')}")
        print("--- End of Group List ---")

        # Optionally, print the full raw JSON structure of all groups
        print("\n--- Raw JSON Structure of ALL Groups ---")
        print(json.dumps(all_groups_list, indent=2))
        print("--- End of Raw JSON Structure ---")

except Exception as e:
    print(f"An error occurred: {e}")

Attempting to retrieve all Zammad Groups (Departments)...
Retrieved 5 groups.

--- List of Zammad Groups ---
Group ID: 1, Name: Users
Group ID: 2, Name: IT support
Group ID: 3, Name: customer support
Group ID: 4, Name: Product Support
Group ID: 5, Name: Returns and Exchanges
--- End of Group List ---

--- Raw JSON Structure of ALL Groups ---
[
  {
    "id": 1,
    "signature_id": 1,
    "email_address_id": 1,
    "name": "Users",
    "name_last": "Users",
    "parent_id": null,
    "assignment_timeout": null,
    "follow_up_possible": "yes",
    "reopen_time_in_days": null,
    "follow_up_assignment": true,
    "active": true,
    "shared_drafts": true,
    "note": "Standard Group/Pool for Tickets.",
    "updated_by_id": 3,
    "created_by_id": 1,
    "created_at": "2025-06-27T12:46:06.391Z",
    "updated_at": "2025-06-27T21:56:31.222Z",
    "user_ids": [
      3
    ],
    "created_by": "-",
    "updated_by": "vi.nce.derm.i.n.a@googlemail.com",
    "email_address": "fwfwe Helpdesk",
 

In [14]:


ticket_params = {
            "title": "requie high  download data with without resolution",
            "group_id": 1,
            "customer_id": customer_id,
            "article": {
                "subject": "requie high  download data",
                "body": " when i download small video it reuiquies lot of amount and the applicaation does not have any resolution option by wwhich i can changes the quality of the video.",
                "type": "email",
                "internal": False,
                # --- THIS IS THE CRITICAL ADDITION ---
                "to": customer_email
            }
        }
ticket_title = ticket_params['title']
ticket_desc = [ticket_params['article']['body']]


In [15]:
from evaluation import ITTicketEvaluator
import os
model_id = os.getenv('HF_MODEL_ID')
api_token = os.getenv('HF_TOKEN')

ticket_evaluator = ITTicketEvaluator(model_id=model_id,api_token=api_token)

result = ticket_evaluator.classify_with_custom_classifier(ticket_desc)
result

Using device: cuda
✅ Custom classifier initialized successfully

=== Custom Classifier Evaluation ===
Classifying 1 queries...
Processing 1/1:  when i download small video it reuiquies lot of a...


[{'query': ' when i download small video it reuiquies lot of amount and the applicaation does not have any resolution option by wwhich i can changes the quality of the video.',
  'department': 'Customer Service',
  'priority': 'low',
  'method': 'custom_classifier'}]